In [1]:
import pandas as pd
import numpy as np
import datatable as dt

In [ ]:
df = dt.fread("datasignal_one_1.txt",sep="\t")

In [ ]:
params = dt.fread(r"home\imperious\ML and CVAE projects\GW from EMRI\gw data\readme\paramlist.txt",sep="\t")
params=params[:250,:]
params

In [1]:
df

NameError: name 'params' is not defined